# 梯度下降

到目前为止，我们构建起了一个最简单的人工神经元网络模型的框架。

模型包括两个部分：参数和逻辑。其中模型逻辑相对固定，模型推理的准确度取决于模型参数是否合理。

那么怎样才能得到合理的模型参数呢？答案是模型可以从海量数据中学习，逐步优化模型参数。这个过程被称为**模型训练**（Model Training）。

## 模型训练

单个人工神经元的本质是线性回归。我们可以通过对损失函数使用**梯度下降**（Gradient Descent）的方法获得模型参数的最优解。

想要模型推理更准确，就是要让损失函数的结果更小。如果把推理函数代入损失函数，就会发现损失函数实际上是关于模型参数的函数。因此可以把损失函数表示为：$J(w)$，其中的$w$是模型参数。

**梯度**就是损失函数在某一点增加最快的方向，实质就是损失函数的偏导数。可以把梯度表示为：$\nabla J(w)$。

**下降**就是让模型参数沿着梯度相反的方向变化，从而达到让损失函数变小的目的：

$$
w_{new} = w_{old} - \nabla J(w_{old})
$$

In [23]:
import numpy as np

## 数据

### 特征、标签

In [24]:
feature = np.array([28.1, 58.0])
label = np.array([165])

## 模型

### 参数：权重、偏差

In [25]:
weight = np.ones([1, 2]) / 2
bias = np.zeros(1)

### 推理函数

In [26]:
def forward(x, w, b):
    return x @ w.T + b

### 损失函数（平均平方差）

In [27]:
def mse_loss(p, y):
    return np.mean(np.square(y - p))

### 梯度函数

损失值是关于预测值的函数（损失函数）；预测值是关于模型参数的函数（推理函数）。

根据微积分的链式规则，我们可以分别计算两个函数的偏导数，然后将其相乘，从而获得损失值关于模型参数的偏导数。

损失函数是：$J = (y - \hat{y})^2$，导数是：$\nabla J = -2(y - \hat{y})$。

推理函数是：$P = wx + b$，权重的导数是：$\nabla_w P = x$，偏差的导数是：$\nabla_b P = 1$。

因此：
* 权重的梯度是：$\nabla_w J = \nabla_w P \cdot \nabla J = -2(y - \hat{y}) x$
* 偏差的梯度是：$\nabla_b J = \nabla_b P \cdot \nabla J = -2(y - \hat{y})$

-----------------

梯度函数将计算权重梯度和偏差梯度的共同部分：$-2(y - \hat{y})$。在深度学习中，这个共同部分通常被称为**误差项**（Delta）。它是从损失函数反向传播的第一棒：

$$
\delta = -2(y - \hat{y})
$$

In [28]:
def gradient(p, y):
    return - 2 * (y - p)

### 反向函数

反向函数将根据梯度调整权重和偏差。这种数据从输出数据反向向输入数据方向的流动，被称为**反向传播**（Backpropagation）：

$$
w = w - \delta \cdot x
$$
$$
b = b - \delta \cdot 1
$$

In [29]:
def backward(x, d, w, b):
    w = w - d * x
    b = b - d
    return w, b

## 验证

### 推理

In [30]:
prediction = forward(feature, weight, bias)

print(f'prediction: {prediction}')

prediction: [43.05]


### 评估

In [31]:
error = mse_loss(prediction, label)

print(f'loss: {error}')

loss: 14871.802500000002


## 训练

### 反向传播

现在，我们根据梯度下降的理论来进行第一次的模型训练。

In [32]:
delta = gradient(prediction, label)
weight, bias = backward(feature, delta, weight, bias)

print(f"weight: {weight}")
print(f"bias: {bias}")

weight: [[ 6854.09 14146.7 ]]
bias: [243.9]


经过一次模型训练，权重和偏差都有了明显的变化。效果如何呢？

### 重新评估

In [33]:
prediction = forward(feature, weight, bias)
error = mse_loss(prediction, label)

print(f'loss: {error}')

loss: 1026548766283.6302


不幸的是，损失量严重恶化。这又是问什么呢？